In [15]:
import requests
from google.protobuf import descriptor_pb2, descriptor_pool, message_factory
from google.protobuf.compiler import plugin_pb2
from google.protobuf.descriptor import FileDescriptor
from google.protobuf.message_factory import GetMessageClass
from google.protobuf.descriptor_pool import DescriptorPool
from google.protobuf.message_factory import MessageFactory


# Configuration
KAFKA_BROKER = "redpanda:9092"
SCHEMA_REGISTRY_URL = "http://redpanda:8081"
TOPIC_NAME = "search_requests"
GROUP_ID = "search-consumer-9"

# Kafka Consumer Config
consumer = Consumer({
    "bootstrap.servers": KAFKA_BROKER,
    "group.id": GROUP_ID,
    "auto.offset.reset": "earliest",
})

consumer.subscribe([TOPIC_NAME])

# Fetch Protobuf schema from Schema Registry
def fetch_schema_from_registry():
    schema_subject = "search_requests-value"
    schema_url = f"{SCHEMA_REGISTRY_URL}/subjects/{schema_subject}/versions/latest"

    response = requests.get(schema_url)
    if response.status_code == 200:
        schema_data = response.json()
        print("\n📜 Fetched Protobuf Schema:\n", schema_data["schema"])  # Debug print
        return schema_data["schema"]
    else:
        raise Exception(f"Failed to fetch schema: {response.text}")


# Compile the Protobuf schema dynamically
def compile_proto_in_memory(proto_schema: str):
    # Define a FileDescriptorProto
    file_descriptor_proto = descriptor_pb2.FileDescriptorProto()
    
    # Manually set the file descriptor properties
    file_descriptor_proto.name = "dynamic.proto"
    file_descriptor_proto.syntax = "proto3"
    
    # Extract the message type from the schema
    lines = proto_schema.split("\n")
    message_name = None
    for line in lines:
        if line.strip().startswith("message "):
            message_name = line.strip().split(" ")[1].split("{")[0]
            break
    
    if not message_name:
        raise ValueError("No message type found in schema")

    # Add the message descriptor dynamically
    message_descriptor = file_descriptor_proto.message_type.add()
    message_descriptor.name = message_name
    
    # Register with a DescriptorPool
    pool = DescriptorPool()
    file_descriptor = pool.Add(file_descriptor_proto)
    
    # Get message descriptor dynamically
    message_descriptor = file_descriptor.message_types_by_name[message_name]
    
    return GetMessageClass(message_descriptor)

# Fetch and compile schema
proto_schema = fetch_schema_from_registry()
DynamicMessageClass = compile_proto_in_memory(proto_schema)

if DynamicMessageClass:
    print("✅ DynamicMessageClass successfully created:", DynamicMessageClass)
else:
    print("❌ Failed to create DynamicMessageClass")


# Start consuming messages
print("\n🚀 Waiting for messages...\n")
while True:
    msg = consumer.poll(1.0)  # Poll for messages

    if msg is None:
        continue
    if msg.error():
        print(f"❌ Consumer error: {msg.error()}")
        continue

    print(f"📥 Raw Message Received: {msg.value()}")  # Debug print

    # Deserialize the Protobuf message using the dynamically loaded class
    try:
        dynamic_message = DynamicMessageClass()
        dynamic_message.ParseFromString(msg.value())
        print(f"✅ Parsed Protobuf Message: {dynamic_message}")
    except Exception as e:
        print(f"❌ Error parsing message: {e}")

    # Print the received message as a dictionary
    print(f"🔹 Received: {dynamic_message}")




📜 Fetched Protobuf Schema:
 syntax = "proto3";

message SearchRequest {
  string query = 1;
  int32 page_number = 2;
  int32 results_per_page = 3;
}


✅ DynamicMessageClass successfully created: <class 'SearchRequest'>

🚀 Waiting for messages...

📥 Raw Message Received: b'\n\x08protobuf\x10\x05\x18!'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\x08redpanda\x10\x01\x18+'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\x08redpanda\x10\x07\x18,'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\x05kafka\x10\t\x18,'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\x08redpanda\x10\x02\x182'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\tstreaming\x10\n\x18\x15'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b'\n\x08protobuf\x10\x07\x182'
✅ Parsed Protobuf Message: 
🔹 Received: 
📥 Raw Message Received: b"\n\x08redpanda\x10\t\x18'"
✅ Parsed Protobuf Message: 
🔹 Re

KeyboardInterrupt: 